Installation d'un package via pip
```bash
pip install --proxy=10.150.2.30:8080 wget
```

In [ ]:
Création de la table de destination des requêtes Power BI
```sql
USE [Staging]
GO

IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'MDW.Dim_PowerBIRapportRequetes') AND type in (N'U'))
BEGIN
       DROP TABLE MDW.Dim_PowerBIRapportRequetes
END



CREATE TABLE MDW.Dim_PowerBIRapportRequetes(
	[Id Item] varcher(36) NOT NULL,
	[Libellé du rapport] [nvarchar](425) NOT NULL,
	[Path] [nvarchar](425) NOT NULL,
	[Requêtes] ntext NOT NULL,
	[Date chargement] [datetime2](0) NOT NULL CONSTRAINT DF_Dim_EvenementLocalisation_Date_chargement_GETDATE DEFAULT GETDATE(),
	[User chargement] [varchar](128) NOT NULL CONSTRAINT DF_Dim_EvenementLocalisation_User_chargement_SYSTEM_USER DEFAULT SYSTEM_USER
) ON [FG_MDW]

GO
```

In [1]:
# utils


In [27]:
import requests
import re
from requests_negotiate_sspi import HttpNegotiateAuth
import os
print(os.getcwd())
    
# Pour un rapport donné
url = 'http://sqlinfobi/ReportsPowerBi/api/v2.0/catalogitems(62035377-4c4a-4150-9813-87c4c06e4f0e)/Content/$value'

r = requests.get(url, allow_redirects=True, auth=HttpNegotiateAuth())
#r = requests.get(url, auth=HttpNegotiateAuth())
# retrieve filename
filename = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
# save file
#open(str(filename), 'wb').write(r.content)

open(str(filename), 'wb').write(r.content)


C:\Users\berhaultb\Documents\Python Scripts\Power BI


50800

In [15]:
import requests
import os
import re
import subprocess
from requests_negotiate_sspi import HttpNegotiateAuth
from utils import *

subfolder = 'power_bi_reports'

if not os.path.exists(subfolder):
    os.makedirs(subfolder)
    
def download_pbix(ItemID, Path):
    url = 'http://sqlinfobi/ReportsPowerBi/api/v2.0/catalogitems('+ItemID+')/Content/$value'

    r = requests.get(url, allow_redirects=True, auth=HttpNegotiateAuth())
    # r = requests.get(url, auth=HttpNegotiateAuth())
    # retrieve filename
    # filename = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
    # save file
    # print(subfolder + Path.rsplit('/', 1)[0] + '/' + ItemID + '.zip')
    os.makedirs(subfolder+Path.rsplit('/', 1)[0], exist_ok=True)
    # print(subfolder + Path + '.zip')

    # Suppression d'ancien rapports
    print(('del "./'+ subfolder + Path +'/*"').replace("/", "\\") + " /F /Q")
    check_output(('if exist ./' + subfolder + Path + ' del "./'+ subfolder + Path +'/*" 2>nul').replace("/", "\\") + " /F /Q", shell=True,encoding="437")
    # subprocess.Popen(('del "./'+ subfolder + Path +'/*"').replace("/", "\\") + " /F /Q")
    
    #open(str(filename), 'wb').write(r.content)
    open(subfolder + Path + '.zip', 'wb').write(r.content)
    
    return str(subfolder + Path), str(subfolder + Path + '.zip')   
    

Réinitialisation/Vidange de la table de destination des informations des rapports Power BI

In [5]:
import pyodbc
from subprocess import check_output

con_sqlinfoservice = pyodbc.connect(r'Driver={ODBC Driver 17 for SQL Server};Server=SQLINFOSERVICE;Database=Staging;Trusted_Connection=yes;')

cur_sqlinfoservice = con_sqlinfoservice.cursor()
cur_sqlinfoservice.execute('TRUNCATE TABLE Staging.MDW.Dim_PowerBIRapportRequetes;')
con_sqlinfoservice.close()

Récupération des informations relatives aux rapports Power BI (identifiant, chemin d'accès, nom du rapport) et stockage en base de données

In [41]:
import subprocess
from time import time, strftime

con_sqlinfobi = pyodbc.connect(r'Driver={ODBC Driver 17 for SQL Server};Server=SQLINFOBI;Database=ReportServePowerbi;Trusted_Connection=yes;')
cur_sqlinfobi = con_sqlinfobi.cursor()
cur_sqlinfobi.execute("SELECT ItemID, Path, Name \
  FROM [ReportServePowerbi].[dbo].[Catalog] t  \
  WHERE Type = 13")

"""
CASE Type
       WHEN 1 THEN 'Folder'
       WHEN 2 THEN 'Report Builder'
       WHEN 5 THEN 'Data Source'
       WHEN 7 THEN 'Report Part'
       WHEN 8 THEN 'Shared Dataset'
	   When 13 Then 'Power BI Report'
       ELSE 'Other'
     END AS TypeDescription
"""
print(str(cur_sqlinfobi.rowcount), 'rapports trouvés')
# Traitement des rapports un par un
while 1:
    row = cur_sqlinfobi.fetchone()
    if not row:
        break
        
    print(strftime('%H:%M:%S'), '- Etape 1 : Téléchargement du rapport')
    start_time = time()
    folder, file_zip = download_pbix(row.ItemID, row.Path)
    print('Rapport téléchargé et non de dossier et de fichier récupérés', time_me(time() - start_time), "\n")

    print(strftime('%H:%M:%S'), '- Extraction du fichier DataMashup')
    start_time = time()
    print(('"C:/Program Files (x86)/7-Zip/7z.exe" -aoa e "'+file_zip+'" "-o./'+folder+'" DataMashup -y').replace("/", "\\"))
    check_output(('"C:/Program Files (x86)/7-Zip/7z.exe" -aoa e "'+file_zip+'" "-o./'+folder+'" DataMashup -y').replace("/", "\\"), shell=True,encoding="437")
    print('DataMashup extrait en', time_me(time() - start_time), "\n")

    #check_output(('"C:/Program Files (x86)/7-Zip/7z.exe" -aoa e "'+folder+'/DataMashup" "-o./'+folder+'" Section1.m -r -y').replace("/", "\\"), shell=True,encoding="437")
    
    print(strftime('%H:%M:%S'), '- Etape 3 : Extraction de Section1.m')
    start_time = time()
    print(('Commande : "C:/Program Files (x86)/7-Zip/7z.exe" -aoa e "'+folder+'/DataMashup" "-o./'+folder+'" Section1.m -r -y').replace("/", "\\"))
    # pour pouvoir utiliser la wait méthode
    p = subprocess.Popen(('"C:/Program Files (x86)/7-Zip/7z.exe" -aoa e "'+folder+'/DataMashup" "-o./'+folder+'" Section1.m -r -y').replace("/", "\\"), shell = True)
    # attend la fin de subprocess.Popen() avant de continuer
    p.wait()
    print('Section1.m extrait en', time_me(time() - start_time), "\n")
          
    #print('### Etape 4 : Suppression de DataMashup ###')
    #print('del "'+folder.replace("/", "\\")+'\DataMashup"')
    #check_output('del "'+folder.replace("/", "\\")+'\DataMashup"', shell=True,encoding="437")
    
    print(strftime('%H:%M:%S'), '- Etape 5 : Lecture de Section1.m')
    start_time = time()
    fichier_de_requetes = str(os.getcwd().replace("\\","/")+('/'+folder+'/Section1.m'))
    print("Requêtes récupérées depuis : ",fichier_de_requetes)
    

    f = open(fichier_de_requetes, 'r', encoding='utf-8')
    #contenu du fichier
    file_contents = f.read()
    
    print(u'[Id Item] : '+row.ItemID)
    print(u'[Libellé du rapport] : '+row.Name)
    print(u'[Path] : '+folder)
    print(u'[Requêtes] : '+file_contents)
    """
    [Id Item] : row.ItemID
	[Libellé du rapport] : row.Name
	[Path] : folder
	[Requêtes] : file_contents
    """
    con_sqlinfoservice2 = pyodbc.connect(r'Driver={SQL Server};Server=SQLINFOSERVICE;Database=Staging;Trusted_Connection=yes;')

    cur_sqlinfoservice2 = con_sqlinfoservice2.cursor()   
    
    #utilisee = "1" if 'BI_Dim_Date' in file_contents else "0"
    
    query = 'INSERT INTO Staging.MDW.Dim_PowerBIRapportRequetes([Id Item], [Libellé du rapport] ,[Path], [Requêtes])  \
values(\''+row.ItemID+'\',\''+row.Name+'\',\''+folder+'\',\''+file_contents.replace("'"," ")+'\')'
    
    print('Query : ' + query)
    
    cur_sqlinfoservice2.execute(query)
    con_sqlinfoservice2.commit()
    con_sqlinfoservice2.close()
    
    
con_sqlinfobi.close()

-1 rapports trouvés
11:30:05 - Etape 1 : Téléchargement du rapport
del ".\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 50 sec 582 ms 

11:30:56 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre.zip" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre" DataMashup -y
DataMashup extrait en 624 ms 672 µs 

11:30:57 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre\DataMashup" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre" Section1.m -r -y
Section1.m extrait en 657 ms 239 µs 

11:30:57 - Etape 5 : 

del ".\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_debut\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 2 sec 552 ms 

11:31:00 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_debut.zip" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_debut" DataMashup -y
DataMashup extrait en 644 ms 293 µs 

11:31:00 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_debut\DataMashup" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_debut" Section1.m -r -y
Section1.m extrai

del ".\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_fin\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 57 sec 798 ms 

11:31:59 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_fin.zip" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_fin" DataMashup -y
DataMashup extrait en 628 ms 316 µs 

11:32:00 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_fin\DataMashup" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre_table_buffer_en_fin" Section1.m -r -y
Section1.m extrait en 679 

del ".\power_bi_reports\01 - principal\01 - rapports power bi\dev\bosi\infodecision\berhaultb\interventions bspp cma cri départements\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 399 ms 150 µs 

11:32:01 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - principal\01 - rapports power bi\dev\bosi\infodecision\berhaultb\interventions bspp cma cri départements.zip" "-o.\power_bi_reports\01 - principal\01 - rapports power bi\dev\bosi\infodecision\berhaultb\interventions bspp cma cri départements" DataMashup -y
DataMashup extrait en 635 ms 294 µs 

11:32:01 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - principal\01 - rapports power bi\dev\bosi\infodecision\berhaultb\interventions bspp cma cri départements\DataMashup" "-o.\power_bi_reports\01 - principal\01 - rapports power bi\dev\bosi\infodecision\berhaultb\interventions bspp cma cri départements

del ".\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Rapport infocentre borh\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 274 ms 705 µs 

11:32:02 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Rapport infocentre borh.zip" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Rapport infocentre borh" DataMashup -y
DataMashup extrait en 606 ms 836 µs 

11:32:03 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Rapport infocentre borh\DataMashup" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Rapport infocentre borh" Section1.m -r -y
Section1.m extrait en 633 ms 342 µs 

11:32:04 - Etape 5 : Le

...

Rapport téléchargé et non de dossier et de fichier récupérés 190 ms 720 µs 

11:38:57 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\03 - BORH\TEST MAJ AUTO.zip" "-o.\power_bi_reports\03 - BORH\TEST MAJ AUTO" DataMashup -y
DataMashup extrait en 616 ms 534 µs 

11:38:58 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\03 - BORH\TEST MAJ AUTO\DataMashup" "-o.\power_bi_reports\03 - BORH\TEST MAJ AUTO" Section1.m -r -y
Section1.m extrait en 616 ms 958 µs 

11:38:59 - Etape 5 : Lecture de Section1.m
Requêtes récupérées depuis :  C:/Users/berhaultb/Documents/Python Scripts/Power BI/power_bi_reports/03 - BORH/TEST MAJ AUTO/Section1.m
[Id Item] : AA523915-37A1-459F-832E-E1B1E6233032
[Libellé du rapport] : TEST MAJ AUTO
[Path] : power_bi_reports/03 - BORH/TEST MAJ AUTO
[Requêtes] : section Section1;

shared Requête1 = let
    Source = Sql.Database("SQLSIRHIUS", "Sirhius", [Query="SEL

del ".\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200402_Synthèse_suiviPM_DG du 7 avril 2020\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 376 ms 566 µs 

11:39:01 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200402_Synthèse_suiviPM_DG du 7 avril 2020.zip" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200402_Synthèse_suiviPM_DG du 7 avril 2020" DataMashup -y
DataMashup extrait en 616 ms 450 µs 

11:39:01 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200402_Synthèse_suiviPM_DG du 7 avril 2020\DataMashup" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200402_Synthèse_suiviPM_DG du 7 avril 2020" Section1.m -r -y
Section1.m extrait en 616 ms 771 µs 

11:39:02 - Etape 5 : Lecture de Section1.m
Requêtes récupérées depui

del ".\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200615_PM_BILAT\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 308 ms 909 µs 

11:39:02 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200615_PM_BILAT.zip" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200615_PM_BILAT" DataMashup -y
DataMashup extrait en 603 ms 996 µs 

11:39:03 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200615_PM_BILAT\DataMashup" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\20200615_PM_BILAT" Section1.m -r -y
Section1.m extrait en 611 ms 130 µs 

11:39:03 - Etape 5 : Lecture de Section1.m
Requêtes récupérées depuis :  C:/Users/berhaultb/Documents/Python Scripts/Power BI/power_bi_reports/05 - BPFB/01 - Rapports Power BI/public/20200615_PM_BILAT/Se

del ".\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport1_situation_EXE_MAND_PPI\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 359 ms 694 µs 

11:39:04 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport1_situation_EXE_MAND_PPI.zip" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport1_situation_EXE_MAND_PPI" DataMashup -y
DataMashup extrait en 627 ms 954 µs 

11:39:04 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport1_situation_EXE_MAND_PPI\DataMashup" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport1_situation_EXE_MAND_PPI" Section1.m -r -y
Section1.m extrait en 620 ms 970 µs 

11:39:05 - Etape 5 : Lecture de Section1.m
Requêtes récupérées depuis :  C:/Users/berhaultb/Documents/Python Scripts/Power BI/power_b

del ".\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport2_situation PROG_EXE_PPI\*" /F /Q
Rapport téléchargé et non de dossier et de fichier récupérés 349 ms 960 µs 

11:39:05 - Extraction du fichier DataMashup
"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport2_situation PROG_EXE_PPI.zip" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport2_situation PROG_EXE_PPI" DataMashup -y
DataMashup extrait en 592 ms 797 µs 

11:39:06 - Etape 3 : Extraction de Section1.m
Commande : "C:\Program Files (x86)\7-Zip\7z.exe" -aoa e "power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport2_situation PROG_EXE_PPI\DataMashup" "-o.\power_bi_reports\05 - BPFB\01 - Rapports Power BI\public\rapport2_situation PROG_EXE_PPI" Section1.m -r -y
Section1.m extrait en 626 ms 916 µs 

11:39:07 - Etape 5 : Lecture de Section1.m
Requêtes récupérées depuis :  C:/Users/berhaultb/Documents/Python Scripts/Power BI/power_b

In [24]:
file_contents = file_contents.replace(" ", "").upper()

if file_contents.find("SQL.DATABASE") != -1:
    file_contents = file_contents.split("SQL.DATABASE(\"")
print(len(file_contents))

['SQLINFOSERVICE","ENTREPOT",[QUERY="SELECTSUBSTRING(CONVERT(VARCHAR,[IDDATELOGAPPEL]),3,6)ASDATETICKET#(LF),YEAR(DATELOGBEGINNINGDT)ASANNEE#(LF),MONTH(DATELOGBEGINNINGDT)ASMOIS#(LF),DAY(DATELOGBEGINNINGDT)ASJOUR#(LF),FORMAT(DATELOGBEGINNINGDT,\'DD-MM-YYY\')ASDATEAPPEL#(LF),FORMAT(DATELOGBEGINNINGDT,\'HH:MM:SS\')ASHEUREAPPEL#(LF),IDDATELOGAPPEL#(LF),CEILING((DURATIONTALKING+DURATIONCALLONHOLD)*0.0166)ASDUREEARRONDI#(LF),CASE#(LF)#(TAB)WHEN(DURATIONTALKING+DURATIONCALLONHOLD)<50THEN0#(LF)#(TAB)ELSEROUND(((DURATIONTALKING+DURATIONCALLONHOLD)/900),0)+1#(LF)ENDASUNITÉSBSPP#(LF),CONVERT(VARCHAR(3),((DURATIONTALKING+DURATIONCALLONHOLD)/3600))+\'H\'+CONVERT(VARCHAR(3),((DURATIONTALKING+DURATIONCALLONHOLD)/60))+\'M\'+CONVERT(VARCHAR(3),((DURATIONTALKING+DURATIONCALLONHOLD)%60))+\'S\'ASDUREEREELLE#(LF)#(LF),PG.LIBELLEPROCESSINGGROUP#(LF),A.LIBELLEAGENT#(LF),DATELOGBEGINNINGTSASEN_BEGINNINGDATE#(LF)FROM[ENTREPOT].[TPH].[FAIT_LOGAPPEL]#(LF)INNERJOINTPH.DIM_PROCESSINGGROUPPGONPG.SIDPROCESSINGGROUP

In [38]:
import pandas as pd
rows = []

for text in file_contents:
    row = [text.split('\"')[0], text.split('\"')[2]] # serveur, base de données
    rows.append(row)
    
df = pd.DataFrame(rows, columns=['Serveur', 'Base de données'])

for index, row in df.groupby(['Serveur', 'Base de données']).size().reset_index(name='Nombre de requêtes').iterrows():
    print(row['Serveur'], row['Base de données'], row['Nombre de requêtes'])

SQLINFOSERVICE ENTREPOT 2


In [48]:
something = 'C:/Users/berhaultb/Documents/Python Scripts/Power BI/power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_debut/Section1.m'
f = open(something, 'r', encoding='utf-8')
file_contents = f.read()
file_contents

'section Section1;\n\nshared #"Départs BSPP" = let\n    Source = Table.Buffer(Excel.Workbook(File.Contents("\\\\bspp.fr\\Travail\\BOSI\\STOU\\Partage\\Section Réalisation des SI\\Groupe Info Décisionnelle\\Données externes PowerBi.xlsx"), null, true)),\n    #"Départs BSPP_Sheet" = Source{[Item="Départs BSPP",Kind="Sheet"]}[Data],\n    #"En-têtes promus" = Table.PromoteHeaders(#"Départs BSPP_Sheet", [PromoteAllScalars=true]),\n    #"Type modifié" = Table.TransformColumnTypes(#"En-têtes promus",{{"Identifiant SAP", Int64.Type}, {"Identifiant défense", type number}, {"Grade act LA", type text}, {"Nom", type text}, {"Prénom", type text}, {"Entrée en service D", type date}, {"Grade act DD", type date}, {"Naissance D", type date}, {"Origine recrutement L", type text}, {"Corps perdant", type text}, {"FE perdante C", type text}, {"FE perdante LA", type text}, {"Catégorie", type text}, {"Situation administrative perdante C", type text}, {"Position gagnante DD", type date}, {"Position statutaire

In [18]:
import subprocess

subprocess.Popen('"C:\Program Files (x86)\7-Zip\7z.exe" -aoa e ".\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre\DataMashup" "-o.\power_bi_reports\01 - Principal\01 - Rapports Power BI\dev\BOSI\Infodecision\BerhaultB\Infocentre" Section1.m -r -y', shell = True)

In [6]:
import subprocess

subprocess.Popen("dir", shell = True)

In [ ]:
con_sqlinfoservice.close()
con_sqlinfoservice2.close()
con_sqlinfobi.close()

In [133]:
import pyodbc 

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=SQLINFOSERVICE;"
                      "Database=Staging;"
                      "Trusted_Connection=yes;")

cursor = cnxn.cursor()
cursor.execute('TRUNCATE TABLE Staging.MDW.Dim_PowerBIRapportRequetes; \
insert into Staging.MDW.Dim_PowerBIRapportRequetes([Id Item], [Libellé du rapport] ,[Path], [Requêtes])  \
values();'\)

b'1\x00.\x001\x007\x00'
b'\xef\xbb\xbf<?xml version="1.0" encoding="utf-8"?><Types xmlns="http://schemas.openxmlformats.org/package/2006/content-types"><Default Extension="JPG" ContentType="" /><Default Extension="json" ContentType="" /><Override PartName="/Version" ContentType="" /><Override PartName="/DataMashup" ContentType="" /><Override PartName="/DiagramLayout" ContentType="" /><Override PartName="/Report/Layout" ContentType="" /><Override PartName="/Settings" ContentType="" /><Override PartName="/Metadata" ContentType="" /><Override PartName="/Report/LinguisticSchema" ContentType="" /><Override PartName="/DiagramState" ContentType="" /><Override PartName="/SecurityBindings" ContentType="" /><Override PartName="/DataModel" ContentType="" /></Types>'
b'\x00\x00\x00\x00\xc1\n'
b'\x00\x00PK\x03\x04\x14\x00\x02\x00\x08\x00\xbcu\x84O\xf5\xc0\xfa\xaa\xab\x00\x00\x00\xfa\x00\x00\x00\x12\x00\x1c\x00Config/Package.xml \xa2\x18\x00(\xa0\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [151]:
from zipfile import ZipFile
import csv


with ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as myzip:
    with myzip.open('DataMashup/Formulas/Section1.m') as myfile:
        mc = myfile.read()
        c = csv.StringIO(mc.decode())
        for row in c:
            print(row)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfd in position 20: invalid start byte

In [156]:
import zipfile
with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip', 'r') as zip_ref:
    zip_ref.extractall('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac')

In [158]:
f = open('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac/DataMashup/Formulas/Section1.m', 'r')
file_contents = f.read()
print (file_contents)

FileNotFoundError: [Errno 2] No such file or directory: 'power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac/DataMashup/Formulas/Section1.m'

In [165]:
import os
import shutil

with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as z:
    with z.open('/DataMashup/[Content_Types].xml') as zf, open('[Content_Types].xml', 'wb') as f:
        shutil.copyfileobj(zf, f)

KeyError: "There is no item named '/DataMashup/[Content_Types].xml' in the archive"

In [168]:
import os

with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as z:
    with open('[Content_Types].xml', 'wb') as f:
        f.write(z.read('DataMashup'))

In [177]:
import zipfile
import io
with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as z:
    with z.open('DataMashup') as z2:
        z2_filedata =  io.BytesIO(z2.read())
        with zipfile.ZipFile(z2_filedata) as nested_zip:
            print( nested_zip.open('Formulas/Section1.m').read())
         

KeyError: "There is no item named 'Formulas/Section1.m' in the archive"

In [61]:
import os
for dirpath, dirnames, filenames in os.walk('.'):
    for f in filenames:
        print(os.path.join(dirpath, f))

.\Récupération des rapports Power BI.ipynb
.\temp.pbix
.\voila
.\.ipynb_checkpoints\Récupération des rapports Power BI-checkpoint.ipynb


In [22]:
import requests

def is_downloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True

print(is_downloadable(url))

AttributeError: 'NoneType' object has no attribute 'lower'

In [23]:
import requests
resp = requests.get(url)

In [25]:
resp.content

b''

In [65]:
import requests
req = requests.get(url)
req

<Response [401]>

In [28]:
import urllib, urlparse

split = urlparse.urlsplit(url)
filename = "C:/Users/berhaultb/Documents/Python Scripts/Power BI" + split.path.split("/")[-1]
urllib.urlretrieve(url, filename)

ModuleNotFoundError: No module named 'urlparse'

In [30]:
from urllib2 import urlopen

f = urlopen(url)
for line in f:
  print len(line)
f.close()

SyntaxError: invalid syntax (<ipython-input-30-391f44a24fd0>, line 5)